In [22]:
import pandas as pd
from database import Database
from datetime import datetime
from Filter.Market import Market

In [29]:
db = Database()
q_data=db.get_finance_report_q()
s_data = db.get_stock('2023/05/16','2023/09/16')
mk_data = db.get_market_index('2023/05/16','2023/09/16')
company_data =db.get_company_basic()

c:\Users\User\Desktop\中央資管\論文\自己的論文\Data_Browsing\database.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_from_db.rename(columns=self.column_mapping, inplace=True)
c:\Users\User\Desktop\中央資管\論文\自己的論文\Data_Browsing\database.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_from_db.rename(columns=self.column_mapping, inplace=True)


In [32]:
 # 將日期列解析為日期時間對象
s_data['日期'] = pd.to_datetime(s_data['日期'])

date_1 = pd.to_datetime('2023/05/16')
date_2 = pd.to_datetime('2023/07/17')
filtered_df = s_data[(s_data['日期'] == date_2)]
# filtered_df = s_data[(s_data['日期'] == date_1) | (s_data['日期'] == date_2)]
# filtered_df = filtered_df.sort_values(by=["證券代碼", "日期"], ascending=[True, False])
# filtered_df = pd.merge(filtered_df, company_data, on='證券代碼', how='left')

filtered_df

,日期,證券代碼,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),5日均價(元),10日均價(元),20日均價(元),60日均價(元),5日均量,10日均量,20日均量,60日均量
62,2023-07-17,1101,37.00,37.10,36.70,36.75,19635.0,36.74,37.10,37.76,38.21,17407.0,19024.0,17341.0,24192.0
377,2023-07-17,1102,43.95,44.15,43.90,43.90,6387.0,43.89,44.07,44.21,44.19,5052.0,4876.0,4876.0,4385.0
692,2023-07-17,1103,21.25,21.50,21.15,21.40,716.0,21.39,21.32,21.65,20.71,815.0,812.0,2267.0,1322.0
1007,2023-07-17,1104,31.00,31.40,30.80,31.15,2705.0,30.34,30.30,30.56,29.31,1567.0,1564.0,1479.0,3209.0
1322,2023-07-17,1108,16.50,16.55,16.30,16.50,696.0,16.47,16.44,16.70,16.10,1175.0,1232.0,1408.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64952,2023-07-17,1786,85.80,90.00,85.60,89.30,1019.0,86.50,87.46,84.72,76.01,752.0,1015.0,1241.0,919.0
65267,2023-07-17,1788,143.00,143.00,142.00,142.00,27.0,141.90,142.00,143.63,144.24,25.0,22.0,32.0,29.0
65582,2023-07-17,1789,30.45,33.25,30.45,33.25,6816.0,30.15,29.63,29.41,27.78,2838.0,2605.0,2840.0,1898.0
65897,2023-07-17,1795,278.50,290.00,278.50,288.50,7336.0,283.80,287.00,304.10,301.83,8680.0,9320.0,11976.0,11481.0


In [ ]:
# price_diff = filtered_df.groupby('證券代碼')['收盤價(元)'].apply(lambda x: x.iloc[0] - x.iloc[1])
# price_diff_percent = filtered_df.groupby('證券代碼')['收盤價(元)'].apply(lambda x: (x.iloc[0] - x.iloc[1])/x.iloc[1])*100
price_diff = filtered_df.groupby('證券代碼')['收盤價(元)'].apply(lambda x: x.iloc[0] - x.iloc[1] if len(x) >= 2 else x.iloc[0])
price_diff_percent = filtered_df.groupby('證券代碼')['收盤價(元)'].apply(lambda x: ((x.iloc[0] - x.iloc[1])/x.iloc[1])*100 if len(x) >= 2 else 0)

result_df = pd.DataFrame({
    '證券代碼': price_diff.index,
    '區間股價變化': price_diff.values,
    '區間股價變化率': price_diff_percent.values
})
# result_df['上市別'] = filtered_df['上市別'].iloc[::2].values
result_df['區間股價變化'] = result_df['區間股價變化'].round(2)
result_df['區間股價變化率'] = result_df['區間股價變化率'].round(2)
result_df = pd.merge(result_df, filtered_df[['證券代碼', '上市別']].drop_duplicates(), on='證券代碼', how='left')

result_df

In [ ]:
#取得大盤表現
market=Market(mk_data,date_1, date_2)
#拆分
tse_df = result_df[result_df['上市別'] == 'TSE']
otc_df = result_df[result_df['上市別'] == 'OTC']
#篩選
tse_filtered = tse_df[tse_df['區間股價變化率'] > market['stock_index_var'].iloc[0]]
otc_filtered = otc_df[otc_df['區間股價變化率'] > market['otc_var'].iloc[0]]

filtered_df = pd.concat([tse_filtered, otc_filtered])
selected_companies=filtered_df['證券代碼'].to_list()
selected_companies